In [1]:
from shared_code.utility.schemas.spark_table_schema import image_table_schema, image_table_schema_with_caption_tokens, tokenize_caption_schema
from shared_code.utility.spark.set_environ import *

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

set_azure_env()
from shared_code.utility.storage.table import TableAdapter
from shared_code.utility.schemas.spark_table_schema import tokenize_caption_schema, image_table_schema_with_caption_tokens
spark = get_session('add-tokens-from-caption')

In [2]:
table_name = "training"
table_adapter: TableAdapter = TableAdapter()
raw_data = table_adapter.get_all_entities(table_name)
spark_df = spark.createDataFrame(list(raw_data), schema=image_table_schema)

spark_df.createOrReplaceTempView("temp_table")

In [3]:
import nltk
from nltk import TweetTokenizer
from nltk import sent_tokenize
from collections import OrderedDict
from pyspark.sql.functions import udf

def get_nlk_tokens(text: str):
	try:
		tokenizer = TweetTokenizer()

		first_sentence = sent_tokenize(text)[0]

		# remove numbers and tokenize the text
		tokenized = tokenizer.tokenize(first_sentence.translate({ord(ch): None for ch in '0123456789'}))

		tokenized = [i for i in tokenized if len(i) > 1]

		tokenized = list(OrderedDict.fromkeys(tokenized))

		pos_tagged_text = nltk.pos_tag(tokenized)

		# Extract all nouns, verbs and adverbs and append to the existing
		prompt_keywords = [i[0] for i in pos_tagged_text if i[1][:2] in ['NN', 'VB', 'RB']]

		prompt_keywords = list(OrderedDict.fromkeys(prompt_keywords))

		return pos_tagged_text
	except Exception as e:
		return None


def get_tokens_for_caption(caption: str):
	try:
		if caption == 'NaN':
			return None
		if caption == "":
			return None
		else:
			tokens = get_nlk_tokens(caption)
			print(tokens)
			return tokens
	except:
		return None

In [4]:
existing_images_not_curated = spark.sql("""
select * from
temp_table
where small_image is not null
	and small_image != ''
	and small_image != 'None'
	and small_image != 'Nan'
	and length(updated_caption)  > 5
	and length(caption) > 5
	and updated_caption not like '%Error%'
	and caption not like '%Error%'
	and exists == True
	and not curated
""", schema=image_table_schema)

In [5]:
get_caption_image_name_udf = udf(get_tokens_for_caption, tokenize_caption_schema)

print("=== Mapping Tokens To Caption ===")
updated_captions_collection = existing_images_not_curated \
	.withColumn("caption_tokens", get_caption_image_name_udf("updated_caption")) \
	.withColumn("text_tokens", get_caption_image_name_udf("text")) \
	.collect()

out_frame = spark.createDataFrame(updated_captions_collection, schema=image_table_schema_with_caption_tokens)

display(out_frame.limit(10).toPandas())

print(f"Total Number Of Records From Pre-Processing {out_frame.count()}")

=== Mapping Tokens To Caption ===


PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "C:\Users\AJ Stangl\AppData\Local\Programs\Python\Python39\lib\socket.py", line 707, in readinto
    raise
socket.timeout: timed out


In [ ]:
print(f" === Writing Records To Cloud parquet ===")
out_frame.write.parquet("D:\\data\\processed\\reddit_images_processed_with_caption_tokens.parquet", mode='overwrite')